<a href="https://colab.research.google.com/github/anhduong77/CustomerSatisfaction/blob/main/notebook/3_Cleaning_data_%26%26_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CustomerSatisfaction/archive/merged_data.csv')

In [ ]:
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
date_cols = [x for x in cat_cols if x.split("_")[-1] in ['timestamp', 'at', 'date']]
cat_cols = [x for x in cat_cols if x not in date_cols]
target = 'review_score'
num_cols.remove(target)

# 1.Data cleaning

In [ ]:
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)
for col in cat_cols:
    df[col].fillna("Unknown", inplace=True)

/tmp/ipython-input-3076652172.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-3076652172.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [ ]:
for col in num_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = df[col].clip(lower, upper)


In [ ]:
def encode_top_categories(df, column, top_n=10):
    # Find top N most frequent categories
    top_values = df[column].value_counts().nlargest(top_n).index

    # Replace rare categories with "Other"
    df[column] = df[column].where(df[column].isin(top_values), other="Other")


    return df


# 2.Feature Engineering

In [ ]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Apply the function to your dataframe
# Assuming your SQL result is stored in 'df'
df['distance_km'] = haversine_distance(
    df['customer_lat'], df['customer_lng'],
    df['seller_lat'], df['seller_lng']
)

# Fill missing distances (some zip codes might be missing) with the median
df['distance_km'] = df['distance_km'].fillna(df['distance_km'].median())

print(df[['customer_state', 'seller_state', 'distance_km']].head())

  customer_state seller_state  distance_km
0             SP           SP    18.576110
1             BA           SP   851.495069
2             GO           SP   514.410666
3             RN           MG  1822.226336
4             SP           SP    29.676625


In [ ]:
df['late_delivery'] = (df['order_delivered_customer_date'] > df['order_estimated_delivery_date']).astype(int)


In [ ]:
df['dissatisfied'] = df[target].apply(lambda x: 1 if x in [1, 2, 3] else 0)

In [ ]:
for col in cat_cols:
    df = encode_top_categories(df, col, top_n=10)


In [ ]:
df.to_csv("/content/drive/MyDrive/CustomerSatisfaction/archive/cleaned_data.csv", index=False)